In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch

import geopandas as gpd
import json
import rasterio

from rasterio import mask

In [3]:
from src.dataloader_epsg_4326 import BridgeDataset

ds=BridgeDataset(1200, use_rnd_pos=True, transform=False)

In [10]:
img, _ = ds[1]
img.shape

torch.Size([7, 48, 48])

### Bad Approach

In [18]:
NUM_CHANNELS=7

data={i:[] for i in range(NUM_CHANNELS)}

for img, _ in tqdm(ds):
    for channel in range(NUM_CHANNELS):
        data[channel].append(img[channel].flatten())

for channel, values in data.items():
    data[channel]=torch.cat(values)
    

100%|██████████| 1684/1684 [00:02<00:00, 578.99it/s]


In [32]:
means=[]
stds=[]
for channel in range(NUM_CHANNELS):
    print("Channel-{ch} mean: {m}\t\tchannel-{ch} standard deviation: {std}".format(
                                ch=channel,
                                m=torch.mean(data[channel]),
                                std=torch.std(data[channel], unbiased=True)
                            ))
    means.append(torch.mean(data[channel]).item())
    stds.append(torch.std(data[channel], unbiased=True).item())

Channel-0 mean: 59002.390625		channel-0 standard deviation: 49175.33203125
Channel-1 mean: 236.64964294433594		channel-1 standard deviation: 15.285372734069824
Channel-2 mean: 234.1954345703125		channel-2 standard deviation: 14.796076774597168
Channel-3 mean: 228.14276123046875		channel-3 standard deviation: 15.021673202514648
Channel-4 mean: 206.40219116210938		channel-4 standard deviation: 18.009830474853516
Channel-5 mean: 214.48724365234375		channel-5 standard deviation: 29.860050201416016
Channel-6 mean: 134.92025756835938		channel-6 standard deviation: 824.8440551757812


In [33]:
tuple(means)

(59002.390625,
 236.64964294433594,
 234.1954345703125,
 228.14276123046875,
 206.40219116210938,
 214.48724365234375,
 134.92025756835938)

In [34]:
tuple(stds)

(49175.33203125,
 15.285372734069824,
 14.796076774597168,
 15.021673202514648,
 18.009830474853516,
 29.860050201416016,
 824.8440551757812)

### Load full images to find stats



In [27]:
def  get_data_stats(data_desc, path, read_channel, exclude_val=None):
    data=rasterio.open(path)
    data=data.read(read_channel).flatten()
    if exclude_val is not None:
        data=data[data!=exclude_val]
    mean=data.mean()
    std=data.std()
    print("Data: {} \t channel: {}".format(data_desc, read_channel))
    print("Mean: {} \t Standard Deviation: {}\n\n".format(mean, std))

In [28]:
get_data_stats(
        data_desc = "population_Rwanda",
        path = "data/population/Rwanda_population_data_2020_UNadj_constrained.tif",
        read_channel = 1,
        exclude_val = -99999
    )

get_data_stats(
        data_desc = "population_Uganda",
        path = "data/population/Uganda_population_data_2020_UNadj_constrained.tif",
        read_channel = 1,
        exclude_val = -99999
    )

Data: population_Rwanda 	 channel: 1
Mean: 15.15371036529541 	 Standard Deviation: 17.416013717651367


Data: population_Uganda 	 channel: 1
Mean: 11.360522270202637 	 Standard Deviation: 15.073326110839844




In [29]:
for i in [1,2,3]:
    get_data_stats(
            data_desc = "OSM Rwanda",
            path = "data/osm/imgs/rwanda_osm_nolab_1-50000_4326.tiff",
            read_channel = i
    )
    
for i in [1,2,3]:
    get_data_stats(
            data_desc = "OSM Uganda",
            path = "data/osm/imgs/uganda_train_osm_nolab_1-50000_4326.tiff",
            read_channel = i
    )

Data: OSM Rwanda 	 channel: 1
Mean: 231.70203462324238 	 Standard Deviation: 21.524828241358563


Data: OSM Rwanda 	 channel: 2
Mean: 233.31144228855203 	 Standard Deviation: 14.25528330439599


Data: OSM Rwanda 	 channel: 3
Mean: 223.8425571862376 	 Standard Deviation: 20.398400049322305


Data: OSM Uganda 	 channel: 1
Mean: 219.18073675126539 	 Standard Deviation: 32.530411532389145


Data: OSM Uganda 	 channel: 2
Mean: 227.8921034703596 	 Standard Deviation: 16.082030009173867


Data: OSM Uganda 	 channel: 3
Mean: 209.27908261428954 	 Standard Deviation: 35.679358964571314




In [31]:
get_data_stats(
        data_desc = "Elevation_Rwanda",
        path = "data/slope_elevation/elevation_rwanda.tif",
        read_channel = 2,
        exclude_val = 0
    )

get_data_stats(
        data_desc = "Elevation_Uganda",
        path = "data/slope_elevation/elevation_uganda.tif",
        read_channel = 2,
        exclude_val = 0
    )

Data: Elevation_Rwanda 	 channel: 2
Mean: 207.0415114662796 	 Standard Deviation: 9.887098411667573


Data: Elevation_Uganda 	 channel: 2
Mean: 222.35360271819636 	 Standard Deviation: 8.056689857889882




In [32]:
get_data_stats(
        data_desc = "Slope_Rwanda",
        path = "data/slope_elevation/slope_rwanda.tif",
        read_channel = 2,
        exclude_val = 0
    )

get_data_stats(
        data_desc = "Slope_Uganda",
        path = "data/slope_elevation/slope_uganda.tif",
        read_channel = 2,
        exclude_val = 0
    )

Data: Slope_Rwanda 	 channel: 2
Mean: 221.30385049308214 	 Standard Deviation: 25.86962156126275


Data: Slope_Uganda 	 channel: 2
Mean: 243.40193346988596 	 Standard Deviation: 15.516082278007305




In [24]:
get_data_stats(
        data_desc = "roads_Rwanda",
        path = "data/osm/roads/rwanda-osm-roads.tif",
        read_channel = 1,
        exclude_val = None
    )

get_data_stats(
        data_desc = "roads_Uganda",
        path = "data/osm/roads/uganda-osm-roads.tif",
        read_channel = 1,
        exclude_val = None
    )

Data: roads_Rwanda 	 channel: 1
Mean: 0.1574539505484869 	 Standard Deviation: 0.9817562848256564


Data: roads_Uganda 	 channel: 1
Mean: 0.08281665629929377 	 Standard Deviation: 0.7271337974582561




In [25]:
get_data_stats(
        data_desc = "waterways_Rwanda",
        path = "data/osm/waterways/rwanda-osm-waterways.tif",
        read_channel = 1,
        exclude_val = None
    )

get_data_stats(
        data_desc = "waterways_Uganda",
        path = "data/osm/waterways/uganda-osm-waterways.tif",
        read_channel = 1,
        exclude_val = None
    )

Data: waterways_Rwanda 	 channel: 1
Mean: 0.016318895533805403 	 Standard Deviation: 0.2068613878439112


Data: waterways_Uganda 	 channel: 1
Mean: 0.003993281769442087 	 Standard Deviation: 0.10063702403324558


